# Applied DataScience Capstone Assignment


## 1) Dataframe Scraped from Wikipedia

In [1]:
import json 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

import folium
import wget
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

tables = pd.read_html(link)
data=tables[0]
data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Editing the dataframe

In [3]:
df=pd.DataFrame(data)

df.drop(df.loc[df['Borough']=='Not assigned'].index,inplace=True)

df['Neighbourhood'].replace(to_replace='Not assigned',value="Queen's Park",inplace=True)

df1=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
df1.shape[0]

103

## 2) Combining dataframes with neighborhood details and location coordinates

In [5]:
Geospatial_Coordinates = wget.download('https://cocl.us/Geospatial_data')

df2=pd.read_csv(r'Geospatial_Coordinates.csv')
df2.head()

df_merge=pd.merge(df1,df2,left_on='Postcode',right_on='Postal Code',how='outer')
df3=df_merge.drop('Postal Code',axis=1)
df3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 3) Segmenting and Clustering

### Exploring the first neighbourhood in the dataframe of Toronto

In [6]:
df_merge=pd.merge(df,df2,left_on='Postcode',right_on='Postal Code',how='outer')
df4=df_merge.drop('Postal Code',axis=1)
df4.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


In [7]:
latitude=df4.loc[0,'Latitude']
longitude=df4.loc[0,'Longitude']

name=df4.loc[0,'Neighbourhood']
print('The latitude and longitude of {} are {} & {}'.format(name,latitude,longitude))

The latitude and longitude of Parkwoods are 43.7532586 & -79.3296565


In [8]:
CLIENT_ID = 'G4JZLP1DYRETG2ND1IMPN1YHLZCERPZR5Y14HH03KDFENEQZ' # your Foursquare ID
CLIENT_SECRET = 'L5XMRZH0YJ0AH4WYRHTWIQUG0IKQGTBN3OR4F5SGMSQKEHBV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G4JZLP1DYRETG2ND1IMPN1YHLZCERPZR5Y14HH03KDFENEQZ
CLIENT_SECRET:L5XMRZH0YJ0AH4WYRHTWIQUG0IKQGTBN3OR4F5SGMSQKEHBV


In [9]:
limit=100
radius=500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    latitude,
    longitude,
    radius,
    limit)

In [10]:
results=requests.get(url).json()


In [11]:
venues=results['response']['groups'][0]['items']
venues=json_normalize(venues)
venues.head()

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4e8d9dcdd5fbbbb6b3003c7b-0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",4e8d9dcdd5fbbbb6b3003c7b,Toronto,CA,Toronto,Canada,245,"[Toronto, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.75197604605557...",43.751976,-79.332140,ON,Brookbanks Park,0,[]
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cb11e2075ebb60cd1c4caad-1,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",4cb11e2075ebb60cd1c4caad,29 Valley Woods Road,CA,Toronto,Canada,312,"[29 Valley Woods Road, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.75197441585782...",43.751974,-79.333114,ON,Variety Store,0,[]


In [12]:
col=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
venues=venues.loc[:,col]
venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.751976,-79.332140
1,Variety Store,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",43.751974,-79.333114


To clean the column of Categories 

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [14]:
venues['venue.categories']=venues.apply(get_category_type,axis=1)
venues.columns=[col.split('.')[-1] for col in venues.columns]
venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [15]:
print('{} venues were returned by Foursquare in Parkwoods'.format(venues.shape[0]))

2 venues were returned by Foursquare in Parkwoods


### To get a map of the venues in Parkwoods

In [16]:
address='Toronto'
geolocator=Nominatim(user_agent='canada_explorer',timeout=100000)
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude

print('Latitude {} & Longitude {} of Toronto'.format(latitude,longitude))

toronto_location=[latitude,longitude]
toronto_location

Latitude 43.653963 & Longitude -79.387207 of Toronto


[43.653963, -79.387207]

In [17]:
Toronto_map=folium.Map(location=toronto_location,zoom_start=11)
Toronto_map 

for lat,lng,name,in zip(venues['lat'],venues['lng'],venues['name']):
    label='{}'.format(name)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat, lng],radius=5,popup=label,color='blue',fill_opacity=0.7,fill=True,parse_html=False).add_to(Toronto_map)
    
Toronto_map    

## 2) Exploring all the neighborhoods in Toronto

Let us explore the neighborhoods of West Toronto,East Toronto,Central Toronto and Downtown Toronto

In [45]:
Toronto_df1=df4[df4['Borough'].isin(['West Toronto','East Toronto','Central Toronto','Downtown Toronto'])]
Toronto_df1.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
12,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
13,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
26,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [46]:
print('No:of neighbourhoods in these boroughs:{}'.format(Toronto_df1.shape[0]))

No:of neighbourhoods in these boroughs:74


In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
       
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [48]:
Toronto_neighborhoods=getNearbyVenues(names=Toronto_df1['Neighbourhood'],
                                      latitudes=Toronto_df1['Latitude'],
                                      longitudes=Toronto_df1['Longitude'])

In [53]:
Toronto_neighborhoods.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [54]:
Toronto_neighborhoods.shape

(3231, 7)

In [55]:
Toronto_neighborhoods.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,17,17,17,17,17,17
Berczy Park,56,56,56,56,56,56
Brockton,23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
CN Tower,17,17,17,17,17,17
Cabbagetown,44,44,44,44,44,44
Central Bay Street,82,82,82,82,82,82
Chinatown,86,86,86,86,86,86


In [56]:
print('There are {} no: of unique venue categories'.format(len(Toronto_neighborhoods['Venue Category'].unique())))


There are 236 no: of unique venue categories


## 3) Analyze each neighborhood in Toronto

In [188]:
toronto_onehot=pd.get_dummies(Toronto_neighborhoods[['Venue Category']],prefix="",prefix_sep="")
toronto_onehot['Neighborhoods']=Toronto_neighborhoods['Neighborhood']

In [64]:
fix_col=[toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot=toronto_onehot[fix_col]
toronto_onehot.head()

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [187]:
toronto_grouped = toronto_onehot.groupby('Neighborhoods').mean().reset_index()
toronto_grouped.head()

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,0.0,0.000000,0.0,0.02,0.0,0.0000,0.000000,0.0,0.000000,0.020000,0.0,0.040000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.02,0.000000,0.01,0.020000,0.0000,0.0,0.01,0.02,0.010000,0.0,0.000000,0.040000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.02,0.000000,0.060000,0.0,0.0,0.0,0.0,0.0,0.01,0.000000,0.0000,0.020000,0.000000,0.03,0.0,0.000000,0.0,0.0,0.0,0.01,0.010000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.000000,0.0100,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0

In [67]:
toronto_grouped.shape

(73, 237)

In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [70]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto_venues_sorted = pd.DataFrame(columns=columns)
Toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    Toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

Toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Bar,Restaurant,Café,Thai Restaurant,Sushi Restaurant,Steakhouse,Cosmetics Shop,Gym,Hotel
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Bar,Plane,Boutique,Rental Car Location,Harbor / Marina,Boat or Ferry
2,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Beer Bar,Bakery,Farmers Market,Restaurant,Cheese Shop,Café,Pharmacy
3,Brockton,Café,Coffee Shop,Breakfast Spot,Gym,Bakery,Performing Arts Venue,Pet Store,Nightclub,Climbing Gym,Restaurant
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Brewery,Smoke Shop,Farmers Market,Spa,Fast Food Restaurant,Burrito Place,Restaurant


## K-Means Clustering

Dataset for clustering

In [71]:
Toronto_clustering=toronto_grouped.drop('Neighborhoods',1)
Toronto_clustering.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,0.0,0.000000,0.0,0.02,0.0,0.0000,0.000000,0.0,0.000000,0.020000,0.0,0.040000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.02,0.000000,0.01,0.020000,0.0000,0.0,0.01,0.02,0.010000,0.0,0.000000,0.040000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.02,0.000000,0.060000,0.0,0.0,0.0,0.0,0.0,0.01,0.000000,0.0000,0.020000,0.000000,0.03,0.0,0.000000,0.0,0.0,0.0,0.01,0.010000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.000000,0.0100,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.0000

In [167]:
num_k=6
kmeans=KMeans(n_clusters=num_k,random_state=123,n_init=25).fit(Toronto_clustering)
labels=kmeans.labels_
labels

array([0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 3, 5,
       3, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 2, 0, 0, 0, 0, 0, 4,
       5, 0, 0, 0, 3, 1, 0, 0, 5, 4, 0, 0, 0, 2, 5, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [168]:
Toronto_venues_sorted['Cluster Labels']=labels

In [170]:
Toronto_venues_sorted.insert(0,'Cluster Labels',labels)
Toronto_venues_sorted.head()


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Adelaide,Coffee Shop,Bar,Restaurant,Café,Thai Restaurant,Sushi Restaurant,Steakhouse,Cosmetics Shop,Gym,Hotel
1,4,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Bar,Plane,Boutique,Rental Car Location,Harbor / Marina,Boat or Ferry
2,0,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Beer Bar,Bakery,Farmers Market,Restaurant,Cheese Shop,Café,Pharmacy
3,0,Brockton,Café,Coffee Shop,Breakfast Spot,Gym,Bakery,Performing Arts Venue,Pet Store,Nightclub,Climbing Gym,Restaurant
4,0,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Brewery,Smoke Shop,Farmers Market,Spa,Fast Food Restaurant,Burrito Place,Restaurant


In [171]:
toronto_df=Toronto_df1
toronto_df=toronto_df.join(Toronto_venues_sorted.set_index('Neighborhood'),on='Neighbourhood',how='inner')
toronto_df.shape

(74, 16)

In [172]:
map_clusters=folium.Map(location=toronto_location,zoom_start=11)

x = np.arange(num_k)

ys=[i+x+(i*x)**2 for i in range(num_k)]

colors_array=cm.rainbow(np.linspace(0,1,len(ys)))

rainbow=[colors.rgb2hex(i) for i in colors_array]

marker_colors=[]

for lat,lng,nei,clus in zip(toronto_df['Latitude'],toronto_df['Longitude'],toronto_df['Neighbourhood'],toronto_df['Cluster Labels']):
    
    label=folium.Popup(str(nei) + ' cluster ' + str(clus), parse_html=True)
    
    folium.CircleMarker([lat,lng],radius=5,popup=label,color=rainbow[clus-1],fill_opacity=0.7).add_to(map_clusters)
    
map_clusters 

In [186]:
Cluster_0 = toronto_df.loc[toronto_df['Cluster Labels'] == 0, toronto_df.columns[[2] + list(range(5, toronto_df.shape[1]))]]
Cluster_0.head()

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Harbourfront,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Theater,Restaurant,Café,Mexican Restaurant,Chocolate Shop
5,Queen's Park,0,Coffee Shop,Gym,Park,Burger Joint,Diner,Portuguese Restaurant,Music Venue,Mexican Restaurant,Juice Bar,Japanese Restaurant
12,Ryerson,0,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Bubble Tea Shop,Pizza Place,Bakery,Bookstore,Diner,Ramen Restaurant
13,Garden District,0,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Bubble Tea Shop,Pizza Place,Bakery,Bookstore,Diner,Ramen Restaurant
26,St. James Town,0,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Breakfast Spot,Clothing Store,Hotel,Park,Beer Bar


In [174]:
Cluster_1 = toronto_df.loc[toronto_df['Cluster Labels'] == 1, toronto_df.columns[[2] + list(range(5, toronto_df.shape[1]))]]
Cluster_1

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
112,Roselawn,1,Garden,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [175]:
Cluster_2 = toronto_df.loc[toronto_df['Cluster Labels'] == 2, toronto_df.columns[[2] + list(range(5, toronto_df.shape[1]))]]
Cluster_2

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
149,Moore Park,2,Playground,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
150,Summerhill East,2,Playground,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [176]:
Cluster_3 = toronto_df.loc[toronto_df['Cluster Labels'] == 3, toronto_df.columns[[2] + list(range(5, toronto_df.shape[1]))]]
Cluster_3

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
121,Forest Hill North,3,Trail,Jewelry Store,Park,Sushi Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
122,Forest Hill West,3,Trail,Jewelry Store,Park,Sushi Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
182,Rosedale,3,Park,Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [177]:
Cluster_4 = toronto_df.loc[toronto_df['Cluster Labels'] == 4, toronto_df.columns[[2] + list(range(5, toronto_df.shape[1]))]]
Cluster_4

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
163,CN Tower,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Bar,Plane,Boutique,Rental Car Location,Harbor / Marina,Boat or Ferry
164,Bathurst Quay,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Bar,Plane,Boutique,Rental Car Location,Harbor / Marina,Boat or Ferry
165,Island airport,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Bar,Plane,Boutique,Rental Car Location,Harbor / Marina,Boat or Ferry
166,Harbourfront West,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Bar,Plane,Boutique,Rental Car Location,Harbor / Marina,Boat or Ferry
167,King and Spadina,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Bar,Plane,Boutique,Rental Car Location,Harbor / Marina,Boat or Ferry
168,Railway Lands,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Bar,Plane,Boutique,Rental Car Location,Harbor / Marina,Boat or Ferry
169,South Niagara,4,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Bar,Plane,Boutique,Rental Car Location,Harbor / Marina,Boat or Ferry


In [178]:
Cluster_5=toronto_df.loc[toronto_df['Cluster Labels'] == 5, toronto_df.columns[[2] + list(range(5, toronto_df.shape[1]))]]
Cluster_5

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
158,Deer Park,5,Coffee Shop,Pub,American Restaurant,Sushi Restaurant,Bagel Shop,Light Rail Station,Fried Chicken Joint,Restaurant,Sports Bar,Supermarket
159,Forest Hill SE,5,Coffee Shop,Pub,American Restaurant,Sushi Restaurant,Bagel Shop,Light Rail Station,Fried Chicken Joint,Restaurant,Sports Bar,Supermarket
160,Rathnelly,5,Coffee Shop,Pub,American Restaurant,Sushi Restaurant,Bagel Shop,Light Rail Station,Fried Chicken Joint,Restaurant,Sports Bar,Supermarket
161,South Hill,5,Coffee Shop,Pub,American Restaurant,Sushi Restaurant,Bagel Shop,Light Rail Station,Fried Chicken Joint,Restaurant,Sports Bar,Supermarket
162,Summerhill West,5,Coffee Shop,Pub,American Restaurant,Sushi Restaurant,Bagel Shop,Light Rail Station,Fried Chicken Joint,Restaurant,Sports Bar,Supermarket


In [185]:
print('The no: of neighborhoods in :\n Cluster0 is {} \n Cluster1 is {} \n Cluster2 is {} \n Cluster3 is {} \n Cluster4 is {} \n Cluster5 is {}'.format(Cluster_0.shape[0],Cluster_1.shape[0],Cluster_2.shape[0],Cluster_3.shape[0],Cluster_4.shape[0],Cluster_5.shape[0]))

The no: of neighborhoods in :
 Cluster0 is 56 
 Cluster1 is 1 
 Cluster2 is 2 
 Cluster3 is 3 
 Cluster4 is 7 
 Cluster5 is 5
